In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]=""
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack.attacker import Attacker

In [ ]:
from allennlp.predictors import Predictor
import allennlp_models.classification
import torch.nn as nn
import textattack
import torch

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/sst-2-basic-classifier-glove-2019.06.27.tar.gz")
        self.model = self.predictor._model
        self.tokenizer = self.predictor._dataset_reader._tokenizer

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

class AllenNLPModelCompressed(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/sst-2-basic-classifier-glove-2019.06.27.tar.gz")
        model = self.predictor._model
        model.cpu()
        quantized_model = torch.quantization.quantize_dynamic(
            model,{nn.Linear,nn.LSTM}, dtype=torch.qint8
        )
        self.predictor._model = quantized_model
        self.model = self.predictor._model
        self.tokenizer = self.predictor._dataset_reader._tokenizer

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]
model_wrapper = AllenNLPModel()
model_wrapper_compressed = AllenNLPModelCompressed()

In [ ]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper_compressed)

attack_args = textattack.AttackArgs(
    num_examples=500,
    random_seed = 42,
    query_budget = 80,
    csv_coloring_style ='plain',
    log_to_csv ="quantized_textfooler_attacks.csv"
)
attacker = Attacker(attack, dataset,attack_args)
res_quant = attacker.attack_dataset()

In [ ]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper)

attack_args = textattack.AttackArgs(
    num_examples=500,
    random_seed = 42,
    query_budget = 80,
    csv_coloring_style ='plain',
    log_to_csv ="main_textfooler_attacks.csv"
)
attacker = Attacker(attack, dataset,attack_args)
res_main = attacker.attack_dataset()

In [ ]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper)
attack_comp = TextFoolerJin2019.build(model_wrapper_compressed)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
main_df = pd.read_csv('main_textfooler_attacks.csv')
quant_df = pd.read_csv('quantized_textfooler_attacks.csv')

In [ ]:
accuracy_score(quant_df['original_output'], quant_df['ground_truth_output'] ),accuracy_score(quant_df['perturbed_output'], quant_df['ground_truth_output'] ), 

In [ ]:
accuracy_score(main_df['original_output'], main_df['ground_truth_output'] ),accuracy_score(main_df['perturbed_output'], main_df['ground_truth_output'] ), 

In [ ]:
main_atts = [x for x in main_df['perturbed_text'].tolist()[:100]]
quant_atts = [x for x in quant_df['perturbed_text'].tolist()[:100]]
labs = [x for x in main_df['ground_truth_output'].tolist()[:100]]

In [ ]:
res = []
for i,x in enumerate(main_atts):
    res.append(attack_comp.attack(x, int(labs[i])))

In [ ]:
c = 0
for x in range(len(res)):
    if 'FAILED' in res[x].goal_function_result_str() or 'SKIPPED' in res[x].goal_function_result_str():
        c+=1
print(100 - c)

In [ ]:
res_q = []
for i,x in enumerate(quant_atts):
    res_q.append(attack.attack(x, int(labs[i])))

In [ ]:
c = 0
for x in range(len(res_q)):
    if 'FAILED' in res_q[x].goal_function_result_str() or 'SKIPPED' in res_q[x].goal_function_result_str():
        c+=1
print(100 - c)